GPAの2値分類

In [62]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [63]:
df=pd.read_csv('../data/Classification-of-students.csv')

カテゴリカルデータtrain_cum.csv validation_num.csvの作成

In [64]:
df.head()

,StudentID,Age,Gender,Ethnicity,ParentalEducation,StudyTimeWeekly,Absences,Tutoring,ParentalSupport,Extracurricular,Sports,Music,Volunteering,GPA,GradeClass
0,1001,17,1,0,2,19.833723,7,1,2,0,0,1,0,2.929196,2.0
1,1002,18,0,0,1,15.408756,0,0,1,0,0,0,0,3.042915,1.0
2,1003,15,0,2,3,4.210570,26,0,2,0,0,0,0,0.112602,4.0
3,1004,17,1,0,3,10.028829,14,0,3,1,0,0,0,2.054218,3.0
4,1005,17,1,0,2,4.672495,17,1,3,0,0,0,0,1.288061,4.0


In [65]:
df['GradeClass']=df['GradeClass'].astype(int)
df['StudyTimeWeekly']=df['StudyTimeWeekly'].astype(int)
df=df.drop('StudentID',axis=1)
df=df.drop('Age',axis=1)
df=df.drop('Absences',axis=1)
df=df.drop('ParentalSupport',axis=1)
df=df.drop('Extracurricular',axis=1)
df=df.drop('Volunteering',axis=1)
df=df.drop('GradeClass',axis=1)

In [66]:
df.loc[df['GPA']<=2,'target']=0
df.loc[(df['GPA']>2)&(df['GPA']<=4),'target']=1

In [67]:
df['target']=df['target'].astype(int)
df=df.drop('GPA',axis=1)

In [68]:
df.head()

,Gender,Ethnicity,ParentalEducation,StudyTimeWeekly,Tutoring,Sports,Music,target
0,1,0,2,19,1,0,1,1
1,0,0,1,15,0,0,0,1
2,0,2,3,4,0,0,0,0
3,1,0,3,10,0,0,0,1
4,1,0,2,4,1,0,0,0


In [69]:
df.shape

(2392, 8)

In [70]:
df['target'].value_counts()

target
0    1274
1    1118
Name: count, dtype: int64

In [71]:
train_num,validation_num=df[:int(len(df)*0.8)],df[int(len(df)*0.8):].reset_index(drop=True)

In [72]:
train_num['target'].value_counts()

target
0    1020
1     893
Name: count, dtype: int64

In [73]:
train_num.head()

,Gender,Ethnicity,ParentalEducation,StudyTimeWeekly,Tutoring,Sports,Music,target
0,1,0,2,19,1,0,1,1
1,0,0,1,15,0,0,0,1
2,0,2,3,4,0,0,0,0
3,1,0,3,10,0,0,0,1
4,1,0,2,4,1,0,0,0


In [74]:
validation_num.head()

,Gender,Ethnicity,ParentalEducation,StudyTimeWeekly,Tutoring,Sports,Music,target
0,1,0,1,12,0,1,0,0
1,1,0,3,8,0,0,0,1
2,1,1,1,1,0,0,1,1
3,1,2,2,9,1,1,0,1
4,0,2,2,14,1,0,0,1


In [75]:
train_num.to_csv('../data/2class_classification_data/train_num.csv',index=False)
validation_num.to_csv('../data/2class_classification_data/validation_num.csv',index=False)

カテゴリカルデータからテキストデータに変換

In [76]:
Gender_mapping={0:'男性は',1:'女性は'}
Ethnicity_mapping={0: '白人で、',1: '黒人で、',2:'アジア人で、',3: '他の人種で、'}
ParentalEducation_mapping={0: '親は中学卒業で、',1: '親は高校卒業で、',2: '親は短期大学もしくは専門学校卒業で、',3: '親は大学卒業で、',4: '親は大学院卒業で、'}
Tutoring_mapping={0:'本人は塾には通っておらず、',1:'本人は塾に通っていて、'}
Sports_mapping={0:'スポーツはしていなくて、',1:'スポーツをしていて、'}
Music_mapping={0:'音楽には無関心だ。',1:'音楽に精通している。'}


In [77]:
df['Gender']=df['Gender'].map(Gender_mapping)
df['Ethnicity']=df['Ethnicity'].map(Ethnicity_mapping)
df['ParentalEducation']=df['ParentalEducation'].map(ParentalEducation_mapping)
df['Tutoring']=df['Tutoring'].map(Tutoring_mapping)
df['Sports']=df['Sports'].map(Sports_mapping)
df['Music']=df['Music'].map(Music_mapping)


In [78]:
text_data = df.apply(lambda row: f"{row['Gender']}{row['Ethnicity']}"+"1週間の勉強時間は"+f"{row['StudyTimeWeekly']}"+"時間で、"+f"{row['ParentalEducation']}{row['Tutoring']}{row['Sports']}{row['Music']}", axis=1)

In [79]:
text_df = pd.DataFrame({'label':df['target'],'sentence': text_data})

In [80]:
train,validation=text_df[:int(len(text_data)*0.8)],text_df[int(len(text_data)*0.8):].reset_index(drop=True)

In [81]:
train.head()

,label,sentence
0,1,女性は白人で、1週間の勉強時間は19時間で、親は短期大学もしくは専門学校卒業で、本人は塾に通...
1,1,男性は白人で、1週間の勉強時間は15時間で、親は高校卒業で、本人は塾には通っておらず、スポー...
2,0,男性はアジア人で、1週間の勉強時間は4時間で、親は大学卒業で、本人は塾には通っておらず、スポ...
3,1,女性は白人で、1週間の勉強時間は10時間で、親は大学卒業で、本人は塾には通っておらず、スポー...
4,0,女性は白人で、1週間の勉強時間は4時間で、親は短期大学もしくは専門学校卒業で、本人は塾に通っ...


In [82]:
validation.head()

,label,sentence
0,0,女性は白人で、1週間の勉強時間は12時間で、親は高校卒業で、本人は塾には通っておらず、スポー...
1,1,女性は白人で、1週間の勉強時間は8時間で、親は大学卒業で、本人は塾には通っておらず、スポーツ...
2,1,女性は黒人で、1週間の勉強時間は1時間で、親は高校卒業で、本人は塾には通っておらず、スポーツ...
3,1,女性はアジア人で、1週間の勉強時間は9時間で、親は短期大学もしくは専門学校卒業で、本人は塾に...
4,1,男性はアジア人で、1週間の勉強時間は14時間で、親は短期大学もしくは専門学校卒業で、本人は塾...


In [83]:
train.to_csv('../data/2class_classification_data/train.csv',index=False,encoding='utf-8-sig')
validation.to_csv('../data/2class_classification_data/validation.csv',index=False,encoding='utf-8-sig')